In [1]:
#Importación de librerías
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error
import random
pd.set_option('display.max_columns', 90)

In [2]:
# CWD = Current Working Directory
cwd = os.getcwd()
raw_dir = cwd + "/../data/raw/"
processed_dir = cwd + "/../data/processed/"
final_dir = cwd + "/../data/final/"

In [3]:
#Se crea cada dataframe para cada variable objetivo
dfmatematicas=pd.read_csv(processed_dir + 'datos_listos_matematicas.csv')
dfingles=pd.read_csv(processed_dir + 'datos_listos_ingles.csv')
dfcnaturales=pd.read_csv(processed_dir + 'datos_listos_c_naturales.csv')
dflcritica=pd.read_csv(processed_dir + 'datos_listos_l_critica.csv')
dfsocialesciudadanas=pd.read_csv(processed_dir + 'datos_listos_sociales_ciudadanas.csv')
dfglobal=pd.read_csv(processed_dir + 'datos_listos_global.csv')

# Diseño del Modelo

## Categorización de puntajes
Los puntajes se categorizarán en 2 grupos, tendencia baja y tendencia alta, esto para ayudar a la precisión en las predicciones del modelo, para la variable puntaje global que va desde 0 hasta 500, la primera categoría va desde 0 hasta 300 puntos, y la segunda de 301 hasta los 500 puntos, esto ya que por arriba de los 300 puntos es donde hay más oportunidades para obtener becas o mejor acceso para universidades que exigen un buen puntaje en la prueba.

Para las variables de las temáticas que evalua el examen la división es la siguiente, cada temática se califica de 0 a 100 puntos, entonces para la categoría de tedencia baja va desde los 0 hasta los 55 puntos y para la categoría de tendencia alta va desde los 56 hasta los 100 puntos.

La categoría de tendencia baja se identifica por el número 1 y la categoría de tendencia alta se identifica por el número 2.

In [4]:
#1=Tendencia Baja, 2=Tendencia alta puntaje global
dfglobal['PUNT_GLOBAL'] = np.where(dfglobal['PUNT_GLOBAL'].between(0,300), 1, dfglobal['PUNT_GLOBAL'])
dfglobal['PUNT_GLOBAL'] = np.where(dfglobal['PUNT_GLOBAL'].between(301, 500), 2, dfglobal['PUNT_GLOBAL'])

In [5]:
#1=Tendencia Baja, 2=Tendencia alta puntaje de matemáticas
dfmatematicas['PUNT_MATEMATICAS'] = np.where(dfmatematicas['PUNT_MATEMATICAS'].between(0,55), 1, dfmatematicas['PUNT_MATEMATICAS'])
dfmatematicas['PUNT_MATEMATICAS'] = np.where(dfmatematicas['PUNT_MATEMATICAS'].between(56, 100), 2, dfmatematicas['PUNT_MATEMATICAS'])

In [6]:
#1=Tendencia Baja, 2=Tendencia alta puntaje de inglés
dfingles['PUNT_INGLES'] = np.where(dfingles['PUNT_INGLES'].between(0,55), 1, dfingles['PUNT_INGLES'])
dfingles['PUNT_INGLES'] = np.where(dfingles['PUNT_INGLES'].between(56, 100), 2, dfingles['PUNT_INGLES'])

In [7]:
#1=Tendencia Baja, 2=Tendencia alta puntaje de ciencias naturales
dfcnaturales['PUNT_C_NATURALES'] = np.where(dfcnaturales['PUNT_C_NATURALES'].between(0,55), 1, dfcnaturales['PUNT_C_NATURALES'])
dfcnaturales['PUNT_C_NATURALES'] = np.where(dfcnaturales['PUNT_C_NATURALES'].between(56, 100), 2, dfcnaturales['PUNT_C_NATURALES'])

In [8]:
#1=Tendencia Baja, 2=Tendencia alta puntaje de lectura crítica
dflcritica['PUNT_LECTURA_CRITICA'] = np.where(dflcritica['PUNT_LECTURA_CRITICA'].between(0,55), 1, dflcritica['PUNT_LECTURA_CRITICA'])
dflcritica['PUNT_LECTURA_CRITICA'] = np.where(dflcritica['PUNT_LECTURA_CRITICA'].between(56, 100), 2, dflcritica['PUNT_LECTURA_CRITICA'])

In [9]:
#1=Tendencia Baja, 2=Tendencia alta puntaje de sociales y ciudadanas
dfsocialesciudadanas['PUNT_SOCIALES_CIUDADANAS'] = np.where(dfsocialesciudadanas['PUNT_SOCIALES_CIUDADANAS'].between(0,55), 1, dfsocialesciudadanas['PUNT_SOCIALES_CIUDADANAS'])
dfsocialesciudadanas['PUNT_SOCIALES_CIUDADANAS'] = np.where(dfsocialesciudadanas['PUNT_SOCIALES_CIUDADANAS'].between(56, 100), 2, dfsocialesciudadanas['PUNT_SOCIALES_CIUDADANAS'])

## Distribución de cada categoría de todos los datasets

In [10]:
print("Global:", dfglobal['PUNT_GLOBAL'].value_counts())
print("Matemáticas:", dfmatematicas['PUNT_MATEMATICAS'].value_counts())
print("Inglés:", dfingles['PUNT_INGLES'].value_counts())
print("Naturales: ", dfcnaturales['PUNT_C_NATURALES'].value_counts())
print("L critica: ",dflcritica['PUNT_LECTURA_CRITICA'].value_counts())
print("Sociales: ", dfsocialesciudadanas['PUNT_SOCIALES_CIUDADANAS'].value_counts())

Global: 1    101658
2     24615
Name: PUNT_GLOBAL, dtype: int64
Matemáticas: 1    80028
2    46245
Name: PUNT_MATEMATICAS, dtype: int64
Inglés: 1.0    85912
2.0    40361
Name: PUNT_INGLES, dtype: int64
Naturales:  1    88746
2    37527
Name: PUNT_C_NATURALES, dtype: int64
L critica:  1    69439
2    56834
Name: PUNT_LECTURA_CRITICA, dtype: int64
Sociales:  1    87641
2    38632
Name: PUNT_SOCIALES_CIUDADANAS, dtype: int64


## Pruebas para encontrar los mejores hiperparámetros

In [11]:
#Valores de prueba para encontrar los mejores hiperparámetros
X_pg = dfglobal.drop(['PUNT_GLOBAL'],axis=1)
y_pg = dfglobal['PUNT_GLOBAL']
X_train_pg, X_test_pg, y_train_pg, y_test_pg = train_test_split(X_pg, y_pg, test_size = 0.20, random_state=3)

In [12]:
#Test random forest
param_dist = {'n_estimators': [100, 200, 300, 400, 500],
              'max_depth': [3 , 5, 10, 15, 20]}

# Create a random forest classifier
rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

# Fit the random search object to the data
rand_search.fit(X_train_pg, y_train_pg)
# Create a variable for the best model
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

Best hyperparameters: {'n_estimators': 500, 'max_depth': 15}


In [14]:
#Test gradient boosting
param_dist = {'n_estimators': [100, 200, 300, 400, 500],
              'max_depth': [3 , 5, 10, 15, 20]}

# Create a random forest classifier
gb = GradientBoostingClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

# Fit the random search object to the data
rand_search.fit(X_train_pg, y_train_pg)
# Create a variable for the best model
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

Best hyperparameters: {'n_estimators': 500, 'max_depth': 15}


# Entrenamiento del Modelo

Se entrenará 1 modelo para cada puntaje utilizando los algoritmos de RandomForestClassifier y GradientBoostingClassifier para encontrar el que mejor de resultados

In [11]:
random_pg = RandomForestClassifier(max_depth=15, n_estimators=500, criterion='log_loss')
gb_model_pg = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=15, random_state=5)
random_pm = RandomForestClassifier(max_depth=15, n_estimators=500, criterion='log_loss')
gb_model_pm = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=15, random_state=10)
random_pi = RandomForestClassifier(max_depth=15, n_estimators=500, criterion='log_loss')
gb_model_pi = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=15, random_state=15)
random_pcn = RandomForestClassifier(max_depth=15, n_estimators=500, criterion='log_loss')
gb_model_pcn = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=15, random_state=20)
random_plc = RandomForestClassifier(max_depth=15, n_estimators=500, criterion='log_loss')
gb_model_plc = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=15, random_state=25)
random_psc = RandomForestClassifier(max_depth=15, n_estimators=500, criterion='log_loss')
gb_model_psc = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=15, random_state=30)

In [64]:
#Modelo RandomForestClassifier de puntaje global
random_pg.fit(X_train_pg,y_train_pg)

RandomForestClassifier(criterion='log_loss', max_depth=5, n_estimators=500)

In [65]:
#Modelo RandomForestClassifier de puntaje global
y_predict_rf_pg = random_pg.predict(X_test_pg)

In [51]:
#Modelo GradientBoostingClassifier de puntaje global
gb_model_pg.fit(X_train_pg, y_train_pg)

GradientBoostingClassifier(max_depth=15, n_estimators=500, random_state=5)

In [52]:
#Modelo GradientBoostingClassifier de puntaje global
y_predict_gb_pg = gb_model_pg.predict(X_test_pg)

In [33]:
#División de columnas de la variable objetivo
X_pm = dfmatematicas.drop(['PUNT_MATEMATICAS'],axis=1)
y_pm = dfmatematicas['PUNT_MATEMATICAS']
X_train_pm, X_test_pm, y_train_pm, y_test_pm = train_test_split(X_pm, y_pm, test_size = 0.20, random_state=42)

In [66]:
#Modelo RandomForestClassifier de puntaje de matemáticas
random_pm.fit(X_train_pm,y_train_pm)

RandomForestClassifier(criterion='log_loss', max_depth=5, n_estimators=500)

In [67]:
#Modelo RandomForestClassifier de puntaje de matemáticas
y_predict_rf_pm = random_pm.predict(X_test_pm)

In [25]:
#Modelo GradientBoostingClassifier de puntaje de matemáticas
gb_model_pm.fit(X_train_pm, y_train_pm)

GradientBoostingClassifier(max_depth=15, n_estimators=500, random_state=10)

In [26]:
#Modelo GradientBoostingClassifier de puntaje de matemáticas
y_predict_gb_pm = gb_model_pm.predict(X_test_pm)

In [13]:
#División de columnas de la variable objetivo
X_pi = dfingles.drop(['PUNT_INGLES'],axis=1)
y_pi = dfingles['PUNT_INGLES']
X_train_pi, X_test_pi, y_train_pi, y_test_pi = train_test_split(X_pi, y_pi, test_size = 0.20, random_state=3)

In [14]:
#Modelo RandomForestClassifier de puntaje de inglés
random_pi.fit(X_train_pi,y_train_pi)

RandomForestClassifier(criterion='log_loss', max_depth=5, n_estimators=500)

In [15]:
#Modelo RandomForestClassifier de puntaje de inglés
y_predict_rf_pi = random_pi.predict(X_test_pi)

In [19]:
#Modelo GradientBoostingClassifier de puntaje de inglés
gb_model_pi.fit(X_train_pi, y_train_pi)

GradientBoostingClassifier(max_depth=15, n_estimators=500, random_state=15)

In [31]:
#Modelo GradientBoostingClassifier de puntaje de inglés
y_predict_gb_pi = gb_model_pi.predict(X_test_pi)

In [12]:
#División de columnas de la variable objetivo
X_pcn = dfcnaturales.drop(['PUNT_C_NATURALES'],axis=1)
y_pcn = dfcnaturales['PUNT_C_NATURALES']
X_train_pcn, X_test_pcn, y_train_pcn, y_test_pcn = train_test_split(X_pcn, y_pcn, test_size = 0.20, random_state=3)

In [13]:
#Modelo RandomForestClassifier de puntaje de ciencias naturales
random_pcn.fit(X_train_pcn,y_train_pcn)

RandomForestClassifier(criterion='log_loss', max_depth=15, n_estimators=500)

In [14]:
#Modelo RandomForestClassifier de puntaje de ciencias naturales
y_predict_rf_pcn = random_pcn.predict(X_test_pcn)

In [15]:
#Modelo GradientBoostingClassifier de ciencias naturales
gb_model_pcn.fit(X_train_pcn, y_train_pcn)

GradientBoostingClassifier(max_depth=15, n_estimators=500, random_state=20)

In [16]:
#Modelo GradientBoostingClassifier de ciencias naturales
y_predict_gb_pcn = gb_model_pcn.predict(X_test_pcn)

In [42]:
#División de columnas de la variable objetivo
X_plc = dflcritica.drop(['PUNT_LECTURA_CRITICA'],axis=1)
y_plc = dflcritica['PUNT_LECTURA_CRITICA']
X_train_plc, X_test_plc, y_train_plc, y_test_plc = train_test_split(X_plc, y_plc, test_size = 0.20, random_state=3)

In [72]:
#Modelo RandomForestClassifier de puntaje de lectura crítica
random_plc.fit(X_train_plc,y_train_plc)

RandomForestClassifier(criterion='log_loss', max_depth=5, n_estimators=500)

In [73]:
#Modelo RandomForestClassifier de puntaje de lectura crítica
y_predict_rf_plc = random_plc.predict(X_test_plc)

In [40]:
#Modelo GradientBoostingClassifier de lectura crítica
gb_model_plc.fit(X_train_plc, y_train_plc)

GradientBoostingClassifier(max_depth=15, n_estimators=500, random_state=25)

In [41]:
#Modelo GradientBoostingClassifier de lectura crítica
y_predict_gb_plc = gb_model_plc.predict(X_test_plc)

In [45]:
#División de columnas de la variable objetivo
X_psc = dfsocialesciudadanas.drop(['PUNT_SOCIALES_CIUDADANAS'],axis=1)
y_psc = dfsocialesciudadanas['PUNT_SOCIALES_CIUDADANAS']
X_train_psc, X_test_psc, y_train_psc, y_test_psc = train_test_split(X_psc, y_psc, test_size = 0.20, random_state=3)

In [74]:
#Modelo RandomForestClassifier de puntaje de lectura crítica
random_psc.fit(X_train_psc,y_train_psc)

RandomForestClassifier(criterion='log_loss', max_depth=5, n_estimators=500)

In [75]:
#Modelo RandomForestClassifier de puntaje de lectura crítica
y_predict_rf_psc = random_psc.predict(X_test_psc)

In [45]:
#Modelo GradientBoostingClassifier de lectura crítica
gb_model_psc.fit(X_train_psc, y_train_psc)

GradientBoostingClassifier(max_depth=15, n_estimators=500, random_state=30)

In [46]:
#Modelo GradientBoostingClassifier de lectura crítica
y_predict_gb_psc = gb_model_psc.predict(X_test_psc)

Verificación y Pruebas del Modelo

In [48]:
print("Puntaje Global random forest:")
print(classification_report(y_test_pg, y_predict_rf_pg, zero_division=0))

Puntaje Global random forest:
              precision    recall  f1-score   support

           1       0.84      0.98      0.90     20394
           2       0.69      0.23      0.34      4861

    accuracy                           0.83     25255
   macro avg       0.77      0.60      0.62     25255
weighted avg       0.81      0.83      0.80     25255



In [106]:
print("Puntaje Global gradient boosting:")
print(classification_report(y_test_pg, y_predict_gb_pg, zero_division=0))

Puntaje Global gradient boosting:
              precision    recall  f1-score   support

           1       0.85      0.96      0.90     20394
           2       0.60      0.27      0.37      4861

    accuracy                           0.82     25255
   macro avg       0.72      0.61      0.63     25255
weighted avg       0.80      0.82      0.80     25255



In [49]:
random_pg.score(X_test_pg,y_test_pg)

0.8317956840229658

In [53]:
gb_model_pg.score(X_test_pg,y_test_pg)

0.824509998020194

In [54]:
print("Puntaje Matemáticas random forest:")
print(classification_report(y_test_pm, y_predict_rf_pm, zero_division=0))

Puntaje Matemáticas random forest:
              precision    recall  f1-score   support

           1       0.71      0.90      0.80     16045
           2       0.68      0.36      0.48      9210

    accuracy                           0.71     25255
   macro avg       0.70      0.63      0.64     25255
weighted avg       0.70      0.71      0.68     25255



In [53]:
print("Puntaje Matemáticas gradient boosting:")
print(classification_report(y_test_pm, y_predict_gb_pm, zero_division=0))

Puntaje Matemáticas gradient boosting:
              precision    recall  f1-score   support

           1       0.72      0.84      0.77     16033
           2       0.61      0.43      0.50      9222

    accuracy                           0.69     25255
   macro avg       0.66      0.63      0.64     25255
weighted avg       0.68      0.69      0.68     25255



In [55]:
random_pm.score(X_test_pm,y_test_pm)

0.7067115422688577

In [55]:
gb_model_pm.score(X_test_pm,y_test_pm)

0.6900415759255593

In [56]:
print("Puntaje Inglés random forest:")
print(classification_report(y_test_pi, y_predict_rf_pi, zero_division=0))

Puntaje Inglés random forest:
              precision    recall  f1-score   support

         1.0       0.75      0.96      0.84     17216
         2.0       0.78      0.31      0.44      8039

    accuracy                           0.75     25255
   macro avg       0.76      0.63      0.64     25255
weighted avg       0.76      0.75      0.71     25255



In [57]:
print("Puntaje Inglés gradient boosting:")
print(classification_report(y_test_pi, y_predict_gb_pi, zero_division=0))

Puntaje Inglés gradient boosting:
              precision    recall  f1-score   support

         1.0       0.77      0.89      0.83     17216
         2.0       0.65      0.44      0.53      8039

    accuracy                           0.75     25255
   macro avg       0.71      0.67      0.68     25255
weighted avg       0.74      0.75      0.73     25255



In [57]:
random_pi.score(X_test_pi,y_test_pi)

0.7518907147099584

In [59]:
gb_model_pi.score(X_test_pi,y_test_pi)

0.748168679469412

In [58]:
print("Puntaje lectura crítica random forest:")
print(classification_report(y_test_plc, y_predict_rf_plc, zero_division=0))

Puntaje lectura crítica random forest:
              precision    recall  f1-score   support

           1       0.65      0.83      0.73     13973
           2       0.68      0.46      0.55     11282

    accuracy                           0.66     25255
   macro avg       0.67      0.64      0.64     25255
weighted avg       0.67      0.66      0.65     25255



In [112]:
print("Puntaje lectura crítica gradient boosting:")
print(classification_report(y_test_plc, y_predict_gb_plc, zero_division=0))

Puntaje lectura crítica gradient boosting:
              precision    recall  f1-score   support

           1       0.68      0.73      0.70     13973
           2       0.63      0.57      0.60     11282

    accuracy                           0.66     25255
   macro avg       0.65      0.65      0.65     25255
weighted avg       0.66      0.66      0.66     25255



In [59]:
random_plc.score(X_test_plc,y_test_plc)

0.6628786378934864

In [63]:
gb_model_plc.score(X_test_plc,y_test_plc)

0.6593941793704217

In [17]:
print("Puntaje ciencias naturtales random forest:")
print(classification_report(y_test_pcn, y_predict_rf_pcn, zero_division=0))

Puntaje ciencias naturtales random forest:
              precision    recall  f1-score   support

           1       0.76      0.94      0.84     17810
           2       0.70      0.30      0.42      7445

    accuracy                           0.76     25255
   macro avg       0.73      0.62      0.63     25255
weighted avg       0.74      0.76      0.72     25255



In [18]:
print("Puntaje ciencias naturtales gradient boosting:")
print(classification_report(y_test_pcn, y_predict_gb_pcn, zero_division=0))

Puntaje ciencias naturtales gradient boosting:
              precision    recall  f1-score   support

           1       0.77      0.90      0.83     17810
           2       0.61      0.36      0.45      7445

    accuracy                           0.74     25255
   macro avg       0.69      0.63      0.64     25255
weighted avg       0.72      0.74      0.72     25255



In [19]:
random_pcn.score(X_test_pcn,y_test_pcn)

0.7554147693526034

In [20]:
gb_model_pcn.score(X_test_pcn,y_test_pcn)

0.7435359334785191

In [60]:
print("Puntaje Sociales random forest:")
print(classification_report(y_test_psc, y_predict_rf_psc, zero_division=0))

Puntaje Sociales random forest:
              precision    recall  f1-score   support

           1       0.73      0.97      0.83     17544
           2       0.72      0.19      0.30      7711

    accuracy                           0.73     25255
   macro avg       0.73      0.58      0.56     25255
weighted avg       0.73      0.73      0.67     25255



In [65]:
print("Puntaje Sociales gradient boosting:")
print(classification_report(y_test_psc, y_predict_gb_psc, zero_division=0))

Puntaje Sociales gradient boosting:
              precision    recall  f1-score   support

           1       0.76      0.90      0.82     17544
           2       0.60      0.35      0.44      7711

    accuracy                           0.73     25255
   macro avg       0.68      0.62      0.63     25255
weighted avg       0.71      0.73      0.71     25255



In [61]:
random_psc.score(X_test_psc,y_test_psc)

0.7298356761037418

In [67]:
gb_model_psc.score(X_test_psc,y_test_psc)

0.7299544644624827

## Resultados de los modelos
Luego de entrenar los modelos, se realizó un reporte del rendimiento de cada modelo, en este caso los modelos entrenados con el algoritmo de random forest classifier demostraron tener un mejor rendimiento en cuanto a precisión de las predicciones.
- Puntaje global: Random forest classifier precisión del 83%, gradient boosting classifier precisión del 82%
- Puntaje de matemáticas: Random forest classifier precisión del 71%, gradient boosting classifier precisión del 69%
- Puntaje de inglés: Random forest classifier precisión del 75%, gradient boosting classifier precisión del 74%
- Puntaje de lectura crítica: Random forest classifier precisión del 66%, gradient boosting classifier precisión del 66%
- Puntaje de ciencias naturales: Random forest classifier precisión del 75%, gradient boosting classifier precisión del 74%
- Puntaje sociales y ciudadanas: Random forest classifier precisión del 73%, gradient boosting classifier precisión del 73%

Es por ello que los modelos que se exportarán y serán utilizados en el producto de datos son aquellos entrenados con el algoritmo Random forest classifier.

# Guardar los modelos entrenados para utilizarlos en el producto de datos

In [15]:
import joblib

In [76]:
#Guardar los modelos en pickle files
pg_model_pkl_file = final_dir+"puntaje_global_model.pkl"  
pm_model_pkl_file = final_dir+"puntaje_matematicas_model.pkl"
pi_model_pkl_file = final_dir+"puntaje_ingles_model.pkl"
pcn_model_pkl_file = final_dir+"puntaje_naturales_model.pkl"
plc_model_pkl_file = final_dir+"puntaje_l_critica_model.pkl"
psc_model_pkl_file = final_dir+"puntaje_sociales_ciudadanas_model.pkl"
joblib.dump(value=random_pg, filename=pg_model_pkl_file)
joblib.dump(value=random_pm, filename=pm_model_pkl_file)
joblib.dump(value=random_pi, filename=pi_model_pkl_file)
joblib.dump(value=random_pcn, filename=pcn_model_pkl_file)
joblib.dump(value=random_plc, filename=plc_model_pkl_file)
joblib.dump(value=random_psc, filename=psc_model_pkl_file)

['C:\\Users\\danie\\OneDrive\\Escritorio\\Proyecto ciencia de datos\\src/../data/final/puntaje_sociales_ciudadanas_model.pkl']